<a href="https://colab.research.google.com/github/Malachyiii/TattooGAN/blob/main/Tattoo_StyleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
!git clone https://github.com/NVlabs/stylegan.git
!ls /content/stylegan/

Cloning into 'stylegan'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 89 (delta 0), reused 1 (delta 0), pack-reused 86
Unpacking objects: 100% (89/89), done.
config.py	     LICENSE.txt	    run_metrics.py
dataset_tool.py      metrics		    stylegan-teaser.png
dnnlib		     pretrained_example.py  training
generate_figures.py  README.md		    train.py


In [ ]:
import sys
sys.path.insert(0, "/content/stylegan")
import dnnlib

In [ ]:
#making a new folder in this particular notebook and moving the images together
!mkdir '/content/LocalData'
random_amount = 3000

import os
import shutil
import random
from tqdm import tqdm

#how many images do we have
files = [file for file in os.listdir('/content/drive/MyDrive/Star_Training_Set/Tattoo') if os.path.isfile(os.path.join('/content/drive/MyDrive/Star_Training_Set/Tattoo', file))]

print(len(files))

mkdir: cannot create directory ‘/content/LocalData’: File exists
30521


In [ ]:
for x in tqdm(range(random_amount), desc ='copy images ...'):
    if len(files) == 0:
        break
    else:
        selection = random.randint(0, len(files)-1) 
        file = files.pop(selection)
        shutil.copyfile(os.path.join('/content/drive/MyDrive/Star_Training_Set/Tattoo', file), os.path.join('/content/LocalData', file))


copy images ...: 100%|██████████| 3000/3000 [15:42<00:00,  3.18it/s]


In [ ]:
!pip install python-resize-image

In [ ]:
#!rm '/content/LocalData/scorpion tattoo designimg_478 (1)2.jpg'

In [ ]:
import cv2
from PIL import Image
from resizeimage import resizeimage
path = '/content/LocalData/'
for filename in tqdm(os.listdir(path),desc ='reading images ...'):
  image = Image.open(path+filename)
  image = image.resize((64,64))
  image.save(path+filename, image.format)

reading images ...: 100%|██████████| 3000/3000 [00:01<00:00, 1756.10it/s]


In [ ]:
!python /content/stylegan/dataset_tool.py create_from_images /content/stylegan/datasets/custom-dataset /content/LocalData




Loading images from "/content/LocalData"
Creating dataset "/content/stylegan/datasets/custom-dataset"



/content/stylegan/dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 3000 images.


1. Now go to the stylegan folder and open the train.py file and make the following changes 

  - Replace line no 37 below  
    - from: desc += '-ffhq'; dataset = EasyDict(tfrecord_dir='ffhq'); train.mirror_augment = True   
    - TO: desc += '-/content/stylegan/datasets/custom-dataset'= EasyDict(tfrecord_dir='/content/stylegan/datasets/custom-dataset'); train.mirror_augment = True
  - uncomment line no 46 below # Number of GPUs. and comment line no 49
  - line number 52, train.total_kimg = 25000 is recommended for complete GAN training of 1024x1024 resolution image. I have set it to 5000. Training will stop after this much iterations (5000 takes about 8 hours)
 

2. Before we start the GAN training, couple of more change needs to be done.  we need to tell which pickled pre-trained model will be used for training our own custom dataset. Stylegan also uses inception-v3  so, we need to get the inception_v3_features.pkl as well

  - If you want, change the pickle file in stylegan/metrics/frechet_inception_distance.py

  - It should say:

  - inception = misc.load_pkl('https://drive.google.com/uc?id=1MzTY44rLToO5APn8TZmfR7_ENSe5aZUn') 

  - If you want, you can download a different pckl file from : https://drive.google.com/drive/folders/1MASQyN5m0voPcx7-9K0r5gObhvvPups7 and replace it with:

  - inception = misc.load_pkl(''YOUR G-Drive inception-v3_features.pkl LINK url') 

3. We are now all set to go for training our own stylegan

In [ ]:
! python /content/stylegan/train.py




Creating the run dir root: results
Creating the run dir: results/00000-sgan-/content/stylegan/datasets/custom-dataset-1gpu
Copying files to the run dir
dnnlib: Running training.training_loop.training_loop() on localhost...



Streaming data using training.dataset.TFRecordDataset...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Dataset shape = [3, 64, 64]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...







G                          

In [ ]:
!mkdir /content/drive/MyDrive/Star_StyleGAN
!cp -v -r -u "/content/results/00000-sgan-/content/stylegan/datasets/custom-dataset-1gpu/." /content/drive/MyDrive/Star_StyleGAN

In [ ]:
!kill $(ps aux | awk '{print $2}')